In [1]:
#!pip install cohere gradio PyMuPDF langchain_community hnswlib pypdf langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.9 MB/s eta 0

In [1]:
import cohere
import gradio as gr
import fitz  # PyMuPDF
import os
import hnswlib
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from cohere import ChatMessage


**Initializing Cohere Key**

In [2]:
# Initialize Cohere client
co = cohere.Client("xhZlPSrYzH1FvIq0ZEaGhPjoNWK8LSzAy7Magodp")

**Loading Pdf Files**

In [3]:
def load_pdfs(pdf_directory):
    texts = []
    for filename in os.listdir(pdf_directory):
        if filename.endswith('.pdf'):
            filepath = os.path.join(pdf_directory, filename)
            with fitz.open(filepath) as pdf:
                text = ""
                for page in pdf:
                    text += page.get_text()
                texts.append(text)
    return texts

pdf_texts = load_pdfs('/content/')

**Chunking the documents**

In [4]:
def split_texts(texts, chunk_size=1000, overlap=20):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    documents = [Document(page_content=text, metadata={"source": f"Document_{i}"}) for i, text in enumerate(texts)]
    chunks = splitter.split_documents(documents)
    return [chunk.page_content for chunk in chunks]

pdf_chunks = split_texts(pdf_texts)
len(pdf_chunks)

2325

**Create embeddings for the chunks**

In [5]:
def embed_texts(texts):
    embeddings = co.embed(texts=texts, model='embed-english-v3.0',input_type="search_document").embeddings
    return embeddings

pdf_embeddings = embed_texts(pdf_chunks)

**Store the embedded chunks in a vector storage**

In [6]:
# Ensure the dimensionality is correct
embedding_dimension = len(pdf_embeddings[0])

# Initialize HNSW index with the correct dimension
idx = hnswlib.Index(space="ip", dim=embedding_dimension)
idx.init_index(max_elements=len(pdf_chunks), ef_construction=512, M=64)

# Add items to the index
idx.add_items(pdf_embeddings, list(range(len(pdf_embeddings))))

# Save the index
idx.save_index("pdf_index.bin")

# To load the index
# loaded_idx = hnswlib.Index(space="ip", dim=1024)
# loaded_idx.load_index("pdf_index.bin", max_elements=len(pdf_chunks))


**Method to retrieve from the vector storage and then rerank those retrieved chunks**

In [7]:
def retrieve_and_rerank(query, n=10, k=5):
    query_emb = co.embed(texts=[query], model="embed-english-v3.0", input_type="search_query").embeddings[0]
    relevant_doc_ids = idx.knn_query(query_emb, k=n)[0][0]
    docs_to_rerank = [pdf_chunks[doc_id] for doc_id in relevant_doc_ids]

    rerank_results = co.rerank(
        query=query,
        documents=docs_to_rerank,
        top_n=k,
        model="rerank-english-v3.0",
        return_documents=True
    )

    texts = [item.document.text for item in rerank_results.results]
    reranked_documents = [{"title": f"Document {i+1}", "description": doc} for i, doc in enumerate(texts)]
    return reranked_documents


**The gradio interface**

In [ ]:
import gradio as gr

global_chat_history = []
preamble = "You are an expert assistant having a great knowledge of UNICA services"

def run_chatbot(message):
    response = co.chat(message=message, model="command-r-plus", search_queries_only=True, chat_history=global_chat_history)

    search_queries = [query.text for query in response.search_queries]

    if search_queries:
        print("Retrieving information...", end="")
        documents = []
        for query in search_queries:
            documents.extend(retrieve_and_rerank(query, 10, 5))

        response = co.chat_stream(
            message=message,
            model="command-r-plus",
            documents=documents,
            preamble=preamble,
            chat_history=global_chat_history,
        )
    else:
        response = co.chat_stream(
            message=message,
            model="command-r-plus",
            preamble=preamble,
            chat_history=global_chat_history,
        )

    chatbot_response = ""
    citations = []
    print("\nChatbot:")

    for event in response:
        if event.event_type == "text-generation":
            print(event.text, end="")
            chatbot_response += event.text
        if event.event_type == "stream-end":
            if event.response.citations:
                citations = event.response.citations
                print("\n\nCITATIONS:")
                for citation in event.response.citations:
                    print(citation)
            if event.response.documents:
                print("\nCITED DOCUMENTS:")
                for document in event.response.documents:
                    print(document)

    # Add to chat history
    global_chat_history.extend(
        [ChatMessage(role="USER", message=message),
         ChatMessage(role="CHATBOT", message=chatbot_response)])

    return chatbot_response, citations

def gradio_chat(query):
    if query.lower() == "quit":
        global global_chat_history
        global_chat_history = []  # Clear chat history
        return "", "", gr.HTML("<h1 style='text-align: center; margin-top: 20%;'>Chat has been closed all the chat history has been deleted.</h1>")

    chatbot_response, citations = run_chatbot(query)

    # Extract citations if available
    if citations:
        citations_text = "\n\nCITATIONS:\n" + "\n".join([f"{citation.text} (from {citation.document_ids})" for citation in citations])
    else:
        citations_text = "Not applicable"

    return chatbot_response, citations_text, ""

# Create a custom HTML component for the note
note_html = "<p style='text-align: center; color: gray;'>Type <strong>'quit'</strong> to end the chat.</p>"

interface = gr.Interface(
    fn=gradio_chat,
    inputs=gr.Textbox(label="User Query"),
    outputs=[
        gr.Textbox(label="Response"),
        gr.Textbox(label="Citations"),
        gr.HTML()
    ],
    title="UNICA Chatbot",
    description=note_html
)
interface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://920b1435f0d7a52f91.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)



Chatbot:
Hello! I'm an AI assistant trained to answer your questions about UNICA services. How can I help you today?Retrieving information...
Chatbot:
I don't have a clear description of UNICA, but I can tell you about some of its features. 

UNICA provides term and perpetual licenses categorized under UPP (Unica Power Pack) and a la carte parts. The license comes with one year of validity, and customers will be charged based on the number of units they subscribed for. 

The UNICA Platform supports UTF-8 as the default character set encoding, which allows users to enter data in any language. The platform also provides a common access point and user interface for UNICA applications. 

UNICA applications must be deployed on a dedicated Java virtual machine (JVM) and require knowledge of operating systems, databases, and web application servers for installation.

CITATIONS:
start=107 end=134 text='term and perpetual licenses' document_ids=['doc_1']
start=153 end=175 text='UPP (Unica Powe